# ⚖️ Experiment B: Welfare & Safety Tests (Pillars 2 & 3)

## 🎯 Research Questions

### Pillar 2: The Collectivism Test (Public Goods Game)
**"Do LLMs pay to punish free-riders?"**

**Core Concept:** Test **Inequality Aversion** and **Altruistic Punishment**

**Hypothesis:**
- **"Toxic Kindness"**: LLMs too nice → refuse to punish → free-riding dominates
- **Welfare Gap**: Individual optimum ≠ Social optimum

**Key Metrics:**
- Punishment Rate: % of rounds where agents punish
- Contribution Decline: Rate of free-riding over time
- Welfare Gap: Δ(social welfare - individual payoff)

### Pillar 3: The Safety Test (Volunteer's Dilemma)
**"Who volunteers when everyone can free-ride?"**

**Core Concept:** Test **Diffusion of Responsibility** and **Bystander Effect**

**Hypothesis:**
- **Large Models**: "Strategic waiting" → Higher disaster risk
- **Small Models**: Random but may save the day
- **Efficiency Paradox**: Smarter → More cautious → Less safe

**Key Metrics:**
- Disaster Rate: % of rounds with no volunteer
- Volunteer Distribution: Who sacrifices most?
- Response Time: First-mover patterns


In [ ]:
# 1. Setup
!git clone https://github.com/technoob05/Trembling-Triads.git
%cd Trembling-Triads

# --- ROBUST INSTALL FOR GPT-OSS-120B / UNSLOTH ---
!pip install --upgrade -qqq uv
!uv pip install --system -qqq "unsloth[base] @ git+https://github.com/unslothai/unsloth" "unsloth_zoo" "transformers==4.56.2" bitsandbytes accelerate pandas openai anthropic mistralai python-dotenv

## 👮‍♂️ Part 1: Public Goods Game (Pillar 2)
Comparing PGG Standard vs. PGG with Punishment.

In [ ]:
# 1.1 Standard PGG (No Punishment) - Control Condition
# Hypothesis: Free-riding dominates over time without punishment mechanism
!python triad_experiment.py --game PGG --no-punishment --rounds 100 --models "Qwen2.5-32B" --reasoning --meta-prompt --meta-rounds "1,25,50,75,100"

print("\n📊 PGG Control Complete: Expect contribution decline")

In [ ]:
# 1.2 PGG with Altruistic Punishment - Treatment Condition
# Hypothesis: Punishment deters free-riding and sustains cooperation
!python triad_experiment.py --game PGG --punishment --rounds 100 --models "Qwen2.5-32B" --reasoning --meta-prompt --meta-rounds "1,25,50,75,100"

print("\n📊 PGG Treatment Complete: Expect sustained cooperation")
print("Check reasoning: Do agents justify punishment decisions?")

## 🆘 Part 2: Volunteer's Dilemma (Pillar 3)
Testing the Bystander Effect.

In [ ]:
# 2.1 The Crisis Run - Who Volunteers?
# Hypothesis: Larger models exhibit "strategic waiting" → Higher disaster risk
!python triad_experiment.py --game VD --rounds 100 --models "Qwen2.5-32B" --reasoning --meta-prompt --meta-rounds "1,25,50,75,100"

print("\n📊 VD Complete: Check for disaster rounds (no volunteer)")
print("Analyze reasoning: Do agents wait for others to volunteer?")

---

## 📊 Quick Analysis: Pillars 2 & 3

Preview punishment patterns and volunteer behavior


In [ ]:
import json
import glob
import pandas as pd

print("=" * 80)
print("PILLAR 2: PUBLIC GOODS GAME ANALYSIS")
print("=" * 80)

# Analyze PGG results
pgg_files = glob.glob('experiment_results_PGG_*.json')

if pgg_files:
    for file in pgg_files:
        with open(file, 'r') as f:
            data = json.load(f)
        
        for exp_name, exp_data in data.items():
            if 'ERROR' in exp_name:
                continue
            
            has_punishment = 'punishment' in exp_name.lower() or 'Punish' in str(exp_data)
            history = exp_data['history']
            
            # Calculate metrics
            contributions = 0
            total = 0
            punishment_count = 0
            
            for round_data in history.values():
                for agent in round_data:
                    total += 1
                    if agent['strategy'] == 'Contribute':
                        contributions += 1
                    if 'punished' in agent:
                        punishment_count += 1
            
            contrib_rate = contributions / total if total > 0 else 0
            punish_per_round = punishment_count / len(history) if history else 0
            
            print(f"\n{exp_name[:60]}")
            print(f"  Contribution Rate: {contrib_rate:.1%}")
            print(f"  Punishment/Round: {punish_per_round:.2f}")
else:
    print("No PGG results yet")

print("\n" + "=" * 80)
print("PILLAR 3: VOLUNTEER'S DILEMMA ANALYSIS")
print("=" * 80)

# Analyze VD results
vd_files = glob.glob('experiment_results_VD_*.json')

if vd_files:
    for file in vd_files:
        with open(file, 'r') as f:
            data = json.load(f)
        
        for exp_name, exp_data in data.items():
            if 'ERROR' in exp_name:
                continue
            
            history = exp_data['history']
            disasters = 0
            volunteer_counts = {'Alice': 0, 'Bob': 0, 'Charlie': 0}
            
            for round_data in history.values():
                volunteers_this_round = 0
                for agent in round_data:
                    if agent['strategy'] == 'Volunteer':
                        volunteers_this_round += 1
                        volunteer_counts[agent['agent']] = volunteer_counts.get(agent['agent'], 0) + 1
                
                if volunteers_this_round == 0:
                    disasters += 1
            
            disaster_rate = disasters / len(history) if history else 0
            
            print(f"\n{exp_name[:60]}")
            print(f"  Disaster Rate: {disaster_rate:.1%} ({disasters}/{len(history)} rounds)")
            print(f"  Volunteer Distribution:")
            for name, count in volunteer_counts.items():
                print(f"    {name}: {count} times")
else:
    print("No VD results yet")

print("\n💡 Check reasoning logs to understand decision-making patterns!")
